# 362. Design Hit Counter

**Difficulty:** Medium | **Category:** Design, Queue

---

## Problem

Design a hit counter which counts the number of hits received in the past `5 minutes` (i.e., the past `300` seconds).

Your system should accept a `timestamp` parameter (in seconds granularity), and you may assume that calls are being made to the system in chronological order (i.e., `timestamp` is monotonically increasing). Several hits may arrive at roughly the same time.

Implement the `HitCounter` class:

- **`HitCounter()`** — Initializes the object of the hit counter system.
- **`hit(timestamp: int) -> None`** — Records a hit that happened at `timestamp` (in seconds). Several hits may happen at the same `timestamp`.
- **`getHits(timestamp: int) -> int`** — Returns the number of hits in the past 5 minutes from `timestamp` (i.e., the past `300` seconds).

---

### Examples

```
Input:
["HitCounter", "hit", "hit", "hit", "getHits", "hit", "getHits", "getHits"]
[[], [1], [2], [3], [4], [300], [300], [301]]

Output:
[null, null, null, null, 3, null, 4, 3]

Explanation:
hitCounter = HitCounter()
hitCounter.hit(1)       # hit at timestamp 1
hitCounter.hit(2)       # hit at timestamp 2
hitCounter.hit(3)       # hit at timestamp 3
hitCounter.getHits(4)   # returns 3 (hits at 1, 2, 3)
hitCounter.hit(300)     # hit at timestamp 300
hitCounter.getHits(300) # returns 4 (hits at 1, 2, 3, 300)
hitCounter.getHits(301) # returns 3 (hits at 2, 3, 300)
```

### Constraints

- `1 <= timestamp <= 2 * 10^9`
- All calls are made in chronological order (timestamps are non-decreasing).
- At most `300` calls will be made to `hit` and `getHits`.

### Follow-up

What if the number of hits per second could be extremely large? How would your design handle that?

## Your Solution

Implement the `HitCounter` class below. Do **not** modify the method signatures.

In [10]:
from collections import deque

class HitCounter:

    def __init__(self):
        # TODO: Initialize your data structure(s) here
        self.hits = deque()
        

    def hit(self, timestamp: int) -> None:
        """
        Record a hit at the given timestamp.
        :param timestamp: The current timestamp (in seconds granularity).
        """
        # TODO: Implement this method
        self.hits.append(timestamp)
        

    def getHits(self, timestamp: int) -> int:
        """
        Return the number of hits in the past 5 minutes (300 seconds).
        :param timestamp: The current timestamp (in seconds granularity).
        :return: Number of hits in the past 300 seconds.
        """
        # TODO: Implement this method
        while self.hits and self.hits[0] <= (timestamp - 300):
            self.hits.popleft()
        return len(self.hits)
        
        

## Quick Test

Run the cell below to verify your solution against the example from the problem statement.

In [11]:
# ── Quick Test (example from problem statement) ──
hc = HitCounter()
hc.hit(1)
hc.hit(2)
hc.hit(3)
assert hc.getHits(4) == 3,   f"Test 1 FAILED: expected 3, got {hc.getHits(4)}"
hc.hit(300)
assert hc.getHits(300) == 4,  f"Test 2 FAILED: expected 4, got {hc.getHits(300)}"
assert hc.getHits(301) == 3,  f"Test 3 FAILED: expected 3, got {hc.getHits(301)}"
print("✅ Quick test passed!")

✅ Quick test passed!


## Submit — Full Test Suite

Run the cell below to evaluate your solution against a comprehensive set of edge-case and stress tests.

In [12]:
import time as _time

def _submit_hit_counter(cls):
    """Comprehensive test suite for HitCounter."""
    passed = 0
    failed = 0
    results = []

    def _run(name, fn):
        nonlocal passed, failed
        try:
            fn()
            passed += 1
            results.append(f"  ✅ {name}")
        except Exception as e:
            failed += 1
            results.append(f"  ❌ {name} — {e}")

    # ── Test 1: Example from problem statement ──
    def test_example():
        hc = cls()
        hc.hit(1); hc.hit(2); hc.hit(3)
        assert hc.getHits(4) == 3
        hc.hit(300)
        assert hc.getHits(300) == 4
        assert hc.getHits(301) == 3
    _run("Example case", test_example)

    # ── Test 2: Single hit ──
    def test_single():
        hc = cls()
        hc.hit(1)
        assert hc.getHits(1) == 1
        assert hc.getHits(300) == 1
        assert hc.getHits(301) == 0
    _run("Single hit", test_single)

    # ── Test 3: Multiple hits at the same timestamp ──
    def test_same_ts():
        hc = cls()
        hc.hit(5); hc.hit(5); hc.hit(5)
        assert hc.getHits(5) == 3
        assert hc.getHits(304) == 3
        assert hc.getHits(305) == 0
    _run("Multiple hits same timestamp", test_same_ts)

    # ── Test 4: All hits expire at once ──
    def test_all_expire():
        hc = cls()
        for t in range(1, 11):
            hc.hit(t)
        assert hc.getHits(10) == 10
        assert hc.getHits(310) == 0
    _run("All hits expire", test_all_expire)

    # ── Test 5: Gradual expiry ──
    def test_gradual():
        hc = cls()
        hc.hit(1); hc.hit(2); hc.hit(100); hc.hit(200); hc.hit(300)
        assert hc.getHits(300) == 5
        assert hc.getHits(301) == 4   # ts=1 expired
        assert hc.getHits(302) == 3   # ts=2 expired
        assert hc.getHits(400) == 2   # ts=100 expired
        assert hc.getHits(500) == 1   # ts=200 expired
        assert hc.getHits(600) == 0   # ts=300 expired
    _run("Gradual expiry", test_gradual)

    # ── Test 6: getHits with no hits recorded ──
    def test_empty():
        hc = cls()
        assert hc.getHits(1) == 0
        assert hc.getHits(300) == 0
    _run("No hits recorded", test_empty)

    # ── Test 7: Hits at boundary (exactly 300s apart) ──
    def test_boundary():
        hc = cls()
        hc.hit(1)
        assert hc.getHits(300) == 1   # 300 - 1 = 299 < 300  → included
        assert hc.getHits(301) == 0   # 301 - 1 = 300 → excluded
    _run("Boundary (300s apart)", test_boundary)

    # ── Test 8: Large timestamp values ──
    def test_large_ts():
        hc = cls()
        base = 2_000_000_000
        hc.hit(base - 100)
        hc.hit(base)
        assert hc.getHits(base) == 2
        assert hc.getHits(base + 200) == 1
        assert hc.getHits(base + 300) == 0
    _run("Large timestamps", test_large_ts)

    # ── Test 9: Rapid sequential hits ──
    def test_rapid():
        hc = cls()
        for t in range(1, 301):
            hc.hit(t)
        assert hc.getHits(300) == 300
        assert hc.getHits(301) == 299
        assert hc.getHits(600) == 0
    _run("300 sequential hits", test_rapid)

    # ── Test 10: Mixed operations – many hits then queries ──
    def test_mixed():
        hc = cls()
        hc.hit(1); hc.hit(1); hc.hit(2); hc.hit(3)
        assert hc.getHits(3) == 4       # all 4 hits within window
        hc.hit(301)
        # ts=1 → 301-1=300 NOT < 300 → excluded (both copies)
        # ts=2 → 301-2=299 < 300 → included
        # ts=3 → included; ts=301 → included → total 3
        assert hc.getHits(301) == 3
        hc.hit(302)
        # ts=2 → 302-2=300 → excluded
        # ts=3, 301, 302 → 3
        assert hc.getHits(302) == 3
    _run("Mixed hits and queries", test_mixed)

    # ── Test 11: Stress test – many duplicate hits ──
    def test_stress_dupes():
        hc = cls()
        for _ in range(300):
            hc.hit(50)
        assert hc.getHits(50) == 300
        assert hc.getHits(349) == 300
        assert hc.getHits(350) == 0
    _run("Stress — 300 duplicate-timestamp hits", test_stress_dupes)

    # ── Performance benchmark ──
    hc = cls()
    t0 = _time.perf_counter()
    for t in range(1, 301):
        hc.hit(t)
    for t in range(1, 301):
        hc.getHits(t)
    elapsed_ms = (_time.perf_counter() - t0) * 1000

    # ── Report ──
    total = passed + failed
    print("=" * 50)
    print(f"  SUBMIT — Design Hit Counter")
    print(f"  {passed}/{total} tests passed")
    print("=" * 50)
    for r in results:
        print(r)
    print("-" * 50)
    print(f"  ⏱  Benchmark (300 hits + 300 queries): {elapsed_ms:.2f} ms")
    if failed == 0:
        print("\n🎉 ALL TESTS PASSED — Great job!")
    else:
        print(f"\n⚠️  {failed} test(s) failed. Review the failures above.")
    print()

_submit_hit_counter(HitCounter)

  SUBMIT — Design Hit Counter
  11/11 tests passed
  ✅ Example case
  ✅ Single hit
  ✅ Multiple hits same timestamp
  ✅ All hits expire
  ✅ Gradual expiry
  ✅ No hits recorded
  ✅ Boundary (300s apart)
  ✅ Large timestamps
  ✅ 300 sequential hits
  ✅ Mixed hits and queries
  ✅ Stress — 300 duplicate-timestamp hits
--------------------------------------------------
  ⏱  Benchmark (300 hits + 300 queries): 0.04 ms

🎉 ALL TESTS PASSED — Great job!

